## UAV-Cyber-SIM

In [1]:
import random
from collections import defaultdict

from config import Color
from helpers import clean
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigQGC,
    ConfigVis,
    Gazebo,
    NoneVisualizer,
    Simulator,
)

clean()

## Create Plans

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=90) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

gcs=[Color.GREEN,Color.ORANGE,Color.RED]
n_uavs_per_gcs = 25
side_len = 10
altitude = 5
max_delay = 10 #sec

base_homes = ENUPose.list([
    (i * 50, j * 3 * side_len, 0, 0)
    for i in range(len(gcs))
    for j in range(n_uavs_per_gcs)
])

base_paths = [
    Plan.create_square_path(side_len=side_len, alt=altitude,heading=0) 
    for _ in base_homes
    ]

colors = [color for color in gcs for _ in range(n_uavs_per_gcs)]

msn_delays=[random.randint(0, max_delay) for _ in base_homes]
msn_delays=[1 for _ in base_homes]


## Assign vehicles to GCS (by color)
gcs_sysids: dict[str, list[int]] = defaultdict(list)

for i,color in enumerate(colors,start=1):
    gcs_sysids[f'{color.name} {color.emoji}'].append(i)

## Gazebo Congiguration

In [3]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway3.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
#gaz_config.show()

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

# No Visualizer

In [5]:
novis_config = ConfigVis[int]()

for i,_ in enumerate(base_homes):
    novis_config.add_vehicle(i)

## Visualization Parameters

In [6]:
novis=NoneVisualizer(novis_config)

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulator

In [7]:
simulator = Simulator(
	visualizers=[qgc],
	gcs_sysids=gcs_sysids,
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=1,
)

orac = simulator.launch()

🚀 ArduPilot SITL vehicle 1 launched (PID 445804)
🚀 ArduPilot SITL vehicle 6 launched (PID 445814)
🚀 ArduPilot SITL vehicle 4 launched (PID 445810)
🚀 ArduPilot SITL vehicle 2 launched (PID 445809)
🚀 ArduPilot SITL vehicle 3 launched (PID 445807)
🚀 ArduPilot SITL vehicle 7 launched (PID 445815)
🚀 UAV logic for vehicle 1 launched (PID 445817)
🚀 ArduPilot SITL vehicle 5 launched (PID 445813)
🚀 UAV logic for vehicle 4 launched (PID 445823)
🚀 UAV logic for vehicle 6 launched (PID 445820)
🚀 ArduPilot SITL vehicle 10 launched (PID 445835)
🚀 UAV logic for vehicle 3 launched (PID 445829)
🚀 UAV logic for vehicle 2 launched (PID 445826)
🚀 Proxy for vehicle 1 launched (PID 445837)
🚀 ArduPilot SITL vehicle 9 launched (PID 445843)
🚀 UAV logic for vehicle 5 launched (PID 445845)
🚀 UAV logic for vehicle 7 launched (PID 445833)
🚀 Proxy for vehicle 4 launched (PID 445846)
🚀 Proxy for vehicle 6 launched (PID 445850)
🚀 ArduPilot SITL vehicle 11 launched (PID 445847)
🚀 ArduPilot SITL vehicle 8 launched (PID

## Oracle checking

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

Vehicle 46: ✅ Sent mission item 0
Vehicle 34: ✅ Sent mission item 0
Vehicle 46: ✅ Sent mission item 1
Vehicle 34: ✅ Sent mission item 1
Vehicle 46: ✅ Sent mission item 2
Vehicle 34: ✅ Sent mission item 2
Vehicle 37: ✅ Sent mission item 0
Vehicle 35: ✅ Sent mission item 0
Vehicle 50: ✅ Sent mission item 0
Vehicle 46: ✅ Sent mission item 3
Vehicle 34: ✅ Sent mission item 3
Vehicle 37: ✅ Sent mission item 1
Vehicle 35: ✅ Sent mission item 1
Vehicle 36: ✅ Sent mission item 0
Vehicle 50: ✅ Sent mission item 1
Vehicle 46: ✅ Sent mission item 4
Vehicle 34: ✅ Sent mission item 4
Vehicle 37: ✅ Sent mission item 2
Vehicle 35: ✅ Sent mission item 2
Vehicle 36: ✅ Sent mission item 1
Vehicle 50: ✅ Sent mission item 2
Vehicle 34: ✅ Sent mission item 5
Vehicle 46: ✅ Sent mission item 5
Vehicle 37: ✅ Sent mission item 3
Vehicle 35: ✅ Sent mission item 3
Vehicle 36: ✅ Sent mission item 2
Vehicle 50: ✅ Sent mission item 3
Vehicle 54: ✅ Sent mission item 0
Vehicle 34: ✅ Sent mission item 6
Vehicle 46: ✅ 

✅ ACK received. DONE message acknowledged.
❎ Vehicle 67 logic stopped.
Connection reset or closed by peer on TCP socket
📤 GCS ← UAV 74: Sending DONE (attempt 10)
❎ Proxy 73 stopped.
📤 GCS ← UAV 70: Sending DONE (attempt 2)
📤 GCS ← UAV 74: Sending DONE (attempt 11)
📤 GCS ← UAV 70: Sending DONE (attempt 3)
✅ ACK received. DONE message acknowledged.
❎ Vehicle 70 logic stopped.
Connection reset or closed by peer on TCP socket
❎ Proxy 67 stopped.
📤 GCS ← UAV 74: Sending DONE (attempt 12)
📤 GCS ← UAV 74: Sending DONE (attempt 13)
❎ Proxy 70 stopped.
📤 GCS ← UAV 74: Sending DONE (attempt 14)
📤 GCS ← UAV 74: Sending DONE (attempt 15)
📤 GCS ← UAV 74: Sending DONE (attempt 16)
📤 GCS ← UAV 74: Sending DONE (attempt 17)
📤 GCS ← UAV 74: Sending DONE (attempt 18)
📤 GCS ← UAV 74: Sending DONE (attempt 19)
📤 GCS ← UAV 74: Sending DONE (attempt 20)
✅ ACK received. DONE message acknowledged.
❎ Vehicle 74 logic stopped.
Connection reset or closed by peer on TCP socket
❎ Proxy 74 stopped.
